In [ ]:
from pprint import pprint
from gensim import utils

import numpy as np

from sklearn.model_selection          import train_test_split

Nesta atividade você irá exercitar o que aprendeu sobre Word Embeddings na disciplina nas seguintes tarefas:
- treinamento de um word embedding em dados jurídicos e avaliação do mesmo em tarefas de similaridade e analogia;
- treinamento de um word embedding em dados de comentários de produtos da Amazon e aplicá-los para classificação de sentimentos.

# 1. Treinando um word embedding com word2vec

Nesta tarefa iremos criar nossos próprios word embeddings. Para isso usaremos dados que são documentos jurídicos coletados da plataforma Jusbrasil.

Para criar nossos embeddings usaremos a classe `Word2Vec` da biblioteca `gensim`.

In [ ]:
!wget https://raw.githubusercontent.com/issilva5/oclsnippets/master/teor_judiciario.txt -O teor_inteiro_jusbrasil.txt
from gensim.models import Word2Vec

--2023-09-29 18:22:42--  https://raw.githubusercontent.com/issilva5/oclsnippets/master/teor_judiciario.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39955780 (38M) [text/plain]
Saving to: ‘teor_inteiro_jusbrasil.txt’

teor_inteiro_jusbra 100%[===================>]  38.10M   204MB/s    in 0.2s    

2023-09-29 18:22:42 (204 MB/s) - ‘teor_inteiro_jusbrasil.txt’ saved [39955780/39955780]



Essa classe recebe em sua inicialização alguns parâmetros importantes:
 - *corpus_file*: o caminho para um arquivo em que cada documento está contido em uma linha.
 - *vector_size*: o tamanho do vetor de embedding a ser gerado.
 - *window_size*: o tamanho da janela a ser considerada no modelo ao buscar por palavras vizinhas.


 Para a atividade você deve explorar **cinco** valores diferentes para pelo menos um desses parâmetros. Isto é, por exemplo, você pode decidir usar vector_size = 50 e testar variar o window_size entre 1 e 5. Ou window_size = 4 e variar o vector_size em [25, 50, 100, 200, 300].

In [ ]:
# Na lista abaixo insira as configurações que você escolheu para testar
# na forma (vector_size, window_size)
combinacoes = [
    (50,5),
    (100,5),
    (200,5),
    (300,5),
    (400,5),
]

Treinaremos então nossos modelos:

In [ ]:
modelos = [Word2Vec(corpus_file='teor_inteiro_jusbrasil.txt', window = ws, vector_size = vs).wv for vs, ws in combinacoes]

Com os modelos treinados acima, você deve responder as seguintes perguntas:

## 1.1 Quais as top-5 palavras mais similares a juiz? Como elas variam considerando as diferentes configurações? Você acha que elas fazem sentido?

In [ ]:
for i, modelo in enumerate(modelos):
  print(f'Modelo {i} - vector_size = {combinacoes[i][0]}, window_size = {combinacoes[i][1]}')
  # Insira aqui o cálculo das palavras mais similares
  # Dica para uma melhor visualização utilize pprint ao invés de print
  top5 = modelo.most_similar(positive=['juiz'], topn=5)
  pprint(top5)
  print('-'*80)

Modelo 0 - vector_size = 50, window_size = 5
[('juíza', 0.7285166382789612),
 ('desembargador', 0.7235047221183777),
 ('(juiz', 0.683124840259552),
 ('magistrado', 0.6386556625366211),
 ('julgador', 0.622808575630188)]
--------------------------------------------------------------------------------
Modelo 1 - vector_size = 100, window_size = 5
[('juíza', 0.6705533266067505),
 ('magistrado', 0.6647987365722656),
 ('desembargador', 0.6407379508018494),
 ('(juiz', 0.621371328830719),
 ('juiz,', 0.580100953578949)]
--------------------------------------------------------------------------------
Modelo 2 - vector_size = 200, window_size = 5
[('juíza', 0.673319399356842),
 ('magistrado', 0.6658952832221985),
 ('desembargador', 0.6235745549201965),
 ('(juiz', 0.5969828963279724),
 ('julgador', 0.5568087697029114)]
--------------------------------------------------------------------------------
Modelo 3 - vector_size = 300, window_size = 5
[('juíza', 0.672717273235321),
 ('magistrado', 0.65538

Nos 5 modelos a palavras foram iguais, mas a ordem foi diferente. Podemos perceber que o valor de similaridade caiu um pouco com relação ao menor valor de vector_size, isso acontece pois estamos aumentando a dimensão do espaço vetorial de cada palavra.

## 1.2 Responda a analogia promotora está para juiz como promotor está para o que? Houve diferença nas respostas considerando os diferentes modelos? Qual deu a melhor resposta?

In [ ]:
for i, modelo in enumerate(modelos):
  print(f'Modelo {i} - vector_size = {combinacoes[i][0]}, window_size = {combinacoes[i][1]}')
  # Insira aqui o cálculo da analogia
  # Dica para uma melhor visualização utilize pprint ao invés de print
  word = modelo.most_similar(positive=['juiz','promotora'], negative=['promotor'], topn=2)
  pprint(word)
  print('-'*80)

Modelo 0 - vector_size = 50, window_size = 5
[('juíza', 0.7319112420082092), ('declinar', 0.6709084510803223)]
--------------------------------------------------------------------------------
Modelo 1 - vector_size = 100, window_size = 5
[('juíza', 0.7099308967590332), ('remetente', 0.5649728178977966)]
--------------------------------------------------------------------------------
Modelo 2 - vector_size = 200, window_size = 5
[('juíza', 0.678337037563324), ('magistrado', 0.5087990164756775)]
--------------------------------------------------------------------------------
Modelo 3 - vector_size = 300, window_size = 5
[('juíza', 0.6888377070426941), ('economica', 0.5222272872924805)]
--------------------------------------------------------------------------------
Modelo 4 - vector_size = 400, window_size = 5
[('juíza', 0.6978282928466797), ('juíza:', 0.5627674460411072)]
--------------------------------------------------------------------------------


A resposta deu igual para os todos modelos: 'juíza'. Mas os modelos apresentaram diferenças na segunda palavra onde só o modelo com um numero mais alto no vector_size resultou em uma palavra quase igual a primeira.

## 1.3 A variação do parâmetro escolhido impactou na qualidade dos modelos gerados? Porquê você acha que esse parâmetro impactou (ou não) a qualidade dos modelos?

Não percebi um impacto significativo nos modelos ao variar o vector_size, pois todos eles compartilham o mesmo conjunto de palavras no top 5 de palavras mais similares a 'juiz'. O que podemos notar são apenas pequenas variações nos valores de similaridade entre os modelos.


## 1.4 Utilize o modelo que você julgar como o melhor para encontrar um caso de palavra em que as palavras mais similares não fazem muito sentido. Por que você acha que o modelo não foi bem neste caso?

In [ ]:
# Insira aqui o código
pprint(modelos[1].most_similar(positive=['tutela'], topn=5))

[('tutela,', 0.7926527261734009),
 ('liminar', 0.7720767855644226),
 ('antecipação', 0.7619683742523193),
 ('urgência', 0.7516860365867615),
 ('urgência,', 0.7032164335250854)]


O modelo identificou 4 palavras bastante distintas em relação à palavra-alvo 'tutela'. Acredito que esse resultado tenha ocorrido devido a palavra 'tutela' poder ser empregada em mais de um contexto, podendo ser usar no sentido de proteção e assistência a um indivíduo ou a responsabilidade sobre algo, como por exemplo, administrar os bens de um menor.


# 2. Usando word embeddings para classificação de sentimento

É possível utilizar word embeddings para classificação de sentimentos através de modelos de linguagem, porém nesta atividade exploraremos uma forma mais simples de usar word embeddings para esta tarefa.

Primeiramente, execute a célula seguinte para gerar o dataset. Os dados que serão utilizados são comentários em produtos no site Amazon.

In [ ]:
!wget https://raw.githubusercontent.com/larifeliciana/books-reviews-portuguese/master/books_pt_neg -O books_pt_neg
!wget https://raw.githubusercontent.com/larifeliciana/books-reviews-portuguese/master/books_pt_pos -O books_pt_pos

corpus_neg = []
corpus_pos = []

for line in open('books_pt_neg'):
  corpus_neg.append(utils.simple_preprocess(line))

for line in open('books_pt_pos'):
  corpus_pos.append(utils.simple_preprocess(line))

--2023-09-29 21:03:53--  https://raw.githubusercontent.com/larifeliciana/books-reviews-portuguese/master/books_pt_neg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 315761 (308K) [text/plain]
Saving to: ‘books_pt_neg’

books_pt_neg        100%[===================>] 308.36K  --.-KB/s    in 0.03s   

2023-09-29 21:03:53 (9.93 MB/s) - ‘books_pt_neg’ saved [315761/315761]

--2023-09-29 21:03:53--  https://raw.githubusercontent.com/larifeliciana/books-reviews-portuguese/master/books_pt_pos
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Lengt

A variável *corpus_neg* contém os comentários negativos, enquanto a variável *corpus_pos* contém os comentários positivos.

In [ ]:
corpus_neg[0]

['concordo',
 'com',
 'outras',
 'avaliações',
 'feitas',
 'entre',
 'outros',
 'sobre',
 'diálogos',
 'fracos',
 'infantis',
 'acrescento',
 'que',
 'li',
 'em',
 'avaliações',
 'de',
 'outros',
 'ebooks',
 'em',
 'português',
 'linguagem',
 'está',
 'pra',
 'lá',
 'de',
 'apelativa',
 'repetitiva',
 'nada',
 'atraente',
 'muito',
 'menos',
 'romântica',
 'perdeu',
 'se',
 'totalmente',
 'noção',
 'não',
 'há',
 'qualquer',
 'critério',
 'de',
 'edição',
 'tentando',
 'agora',
 'os',
 'em',
 'inglês',
 'para',
 'ver',
 'se',
 'tem',
 'algo',
 'melhor']

In [ ]:
corpus_pos[0]

['enfim',
 'final',
 'da',
 'série',
 'chegou',
 'me',
 'deixou',
 'arrebatada',
 'emocionada',
 'um',
 'final',
 'cheio',
 'de',
 'amor',
 'lembranças',
 'perfeito',
 'para',
 'uma',
 'série',
 'tão',
 'emocionante',
 'intensa',
 'mauricio',
 'era',
 'último',
 'dos',
 'lafaietes',
 'solteiro',
 'mas',
 'eleonora',
 'arrebatou',
 'seu',
 'coração',
 'ainda',
 'que',
 'não',
 'estivesse',
 'nos',
 'seus',
 'planos',
 'ele',
 'desde',
 'sempre',
 'soube',
 'que',
 'queria',
 'correu',
 'atrás',
 'para',
 'realizar',
 'os',
 'seus',
 'sonhos',
 'eleonora',
 'não',
 'foi',
 'uma',
 'mocinha',
 'fácil',
 'dócil',
 'maumau',
 'ao',
 'contrário',
 'decidido',
 'intenso',
 'apaixonado',
 'uma',
 'história',
 'cativante',
 'que',
 'vai',
 'te',
 'envolvendo',
 'amor',
 'não',
 'tem',
 'idade',
 'não',
 'vê',
 'traumas',
 'preconceitos',
 'ele',
 'vai',
 'arrebatando',
 'tudo',
 'supera',
 'foi',
 'exatamente',
 'isso',
 'que',
 'autora',
 'foi',
 'nos',
 'revelando',
 'aos',
 'poucos',
 'uma',

Agora para treinar nossos modelos precisamos obter uma representação de cada sentença como um vetor de features, que chamaremos de *sentence embedding*. Nesta atividade iremos gerar os sentence embedding através da média dos word embeddings das palavras que compõem cada sentença.

Primeiramente, crie um novo modelo Word2Vec usando ambos os conjuntos de sentenças, com parâmetros a sua escolha.

In [ ]:
# Além dos parâmetros que discutimos acima, inclua o parâmetro min_count=1
corpus = corpus_neg + corpus_pos
modelo = Word2Vec(sentences=corpus, window = 5, vector_size = 100, min_count=1).wv

Agora vamos criar uma função que dada uma sentença e um modelo Word2Vec retorna o *sentence embedding* dessa sentença. Para calcular a média dos vetores você pode utilizar a função `np.mean`.

In [ ]:
def get_sentence_embedding(sentence, model):
  embeddings = [model[word] for word in sentence if word in model]
  return np.mean(embeddings, axis=0)

A seguir você deve aplicar a função sobre os dados.

In [ ]:
# Calcule a lista com os embeddings das sentenças negativas
sentences_embeddings_neg = []
for sentence in corpus_neg:
  if len(sentence) > 0:
    sentences_embeddings_neg.append(get_sentence_embedding(sentence, modelo))

# Calcule a lista com os embeddings das sentenças positivas
sentences_embeddings_pos = []
for sentence in corpus_pos:
  if len(sentence) > 0:
    sentences_embeddings_pos.append(get_sentence_embedding(sentence, modelo))


In [ ]:
X = np.concatenate([sentences_embeddings_neg, sentences_embeddings_pos])
y = np.concatenate([[-1]*len(sentences_embeddings_neg), [1]*len(sentences_embeddings_pos)])

Crie uma partição treino e teste usando a função `train_test_split` do sklearn, usando as variáveis X e y acima.

In [ ]:
# Crie a partição treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7)

## 2.1 Agora instancie pelo menos dois modelos de classificação, em seguida os treine e avalie na sua partição. Discuta os resultados em termos das métricas previamentes vistas em sala.

In [ ]:
# Treinamento e avaliação do modelo 1
from sklearn.linear_model import LogisticRegression , Perceptron
from sklearn.metrics import accuracy_score, classification_report

model_rl = LogisticRegression(random_state=0).fit(X_train, y_train)

y_pred_rl = model_rl.predict(X_test)

accuracy = accuracy_score(y_test, y_pred_rl)

print("Acurácia da Regressão Logística:", accuracy, "\n")
print(classification_report(y_test, y_pred_rl))

Acurácia da Regressão Logística: 0.5453895639742673 

              precision    recall  f1-score   support

          -1       0.54      0.58      0.56       696
           1       0.55      0.51      0.53       703

    accuracy                           0.55      1399
   macro avg       0.55      0.55      0.55      1399
weighted avg       0.55      0.55      0.54      1399



In [ ]:
# Treinamento e avaliação do modelo 2

model_p = Perceptron().fit(X_train, y_train)

y_pred_p = model_p.predict(X_test)

accuracy = accuracy_score(y_test, y_pred_p)

print("Acurácia do Percerptron:", accuracy, "\n")
print(classification_report(y_test, y_pred_p))

Acurácia do Percerptron: 0.505360972122945 

              precision    recall  f1-score   support

          -1       0.83      0.01      0.01       696
           1       0.50      1.00      0.67       703

    accuracy                           0.51      1399
   macro avg       0.67      0.50      0.34      1399
weighted avg       0.67      0.51      0.34      1399



A média de acurácia dos dois modelos foi baixa, aproximadamente 50%, indicando uma taxa de acertos nas previsões inferior. Isso também é refletido na métrica de previsões, recall e f1-score. No entanto, vale ressaltar que o modelo Perceptron obteve um desempenho ligeiramente superior ao da Regressão Logística.